In [49]:
import tensorflow

In [38]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import pickle
import psycopg2

In [39]:
# Read data from PostGres SQL Data Base
with open("../database/secrets", "r") as file:
    secrets = [i.strip('\n') for i in file.readlines()]


def conn_curs():
    """
    makes a connection to the database dont worry these are dummy keys
    """

    connection = psycopg2.connect(dbname=secrets[4], user=secrets[4],
                                  password=secrets[5], host=secrets[6])
    cursor = connection.cursor()
    return connection, cursor


In [40]:
conn, curs = conn_curs()

In [41]:
df = pd.read_sql("SELECT * FROM posts", conn)

In [42]:
df.head()

,id,text,subreddit
0,1,Thousand Year Blood War Arc Anime Adaptation M...,bleach
1,2,Burn The Witch - Chapter 4 Discussion Thread #...,bleach
2,3,Let the journey begin.,bleach
3,4,"Since Ichigos an English Literature major, thi...",bleach
4,5,I just made these Ulquiorra customs for a clie...,bleach


# Vectorize

In [43]:
le = LabelEncoder()
le.fit(df.subreddit.unique())

LabelEncoder()

In [44]:
le.classes_

array(['Animesuggest', 'AquaSwap', 'AusLegal', 'BackYardChickens',
       'BeardedDragons', 'Bedbugs', 'Beekeeping', 'Berserk',
       'BokuNoHeroAcademia', 'CaptainTsubasaDT', 'CasualConversation',
       'CatAdvice', 'Clairvoyantreadings', 'DDLC', 'DarlingInTheFranxx',
       'DecidingToBeBetter', 'Dogtraining', 'Dragonballsuper',
       'GetMotivated', 'GiftIdeas', 'Gifts', 'Gundam', 'Gunpla',
       'HelpMeFind', 'HunterXHunter', 'INeedAName', 'IWantOut',
       'ImmigrationCanada', 'KissAnime', 'LegalAdviceUK', 'LifeProTips',
       'MLPLounge', 'NameThatSong', 'NoStupidQuestions', 'Petloss',
       'Pets', 'RATS', 'RBI', 'Rabbits', 'ShingekiNoKyojin',
       'StardustCrusaders', 'StopGaming', 'TooAfraidToAsk',
       'WouldYouRather', 'anime', 'answers', 'araragi', 'ask', 'asklaw',
       'ballpython', 'bettafish', 'bleach', 'cats', 'changemyview',
       'christmas', 'datarecovery', 'dbz', 'deathnote', 'declutter',
       'dogs', 'explainlikeimfive', 'fatestaynight', 'ferrets',


In [45]:
X_train, X_test, y_train, y_test = train_test_split(df.text, le.transform(df.subreddit), test_size=0.33, random_state=42)

In [46]:
X_train.tolist()[0]

'Starting to study So my classes only start next week but I am retaking some classes I failed next semester so I thought Id start making resumes and some exercises to pull ahead and not get overwhelmed when I start taking my new classes as well... Wish me luck!'

In [47]:
y_train

array([81, 81, 90, ..., 42, 67, 75])

In [48]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from numpy import array

In [49]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text
 
# source text
data = X_train.tolist()
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
encoded = tokenizer.texts_to_sequences(data)


In [50]:
sequences = encoded

In [51]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# # encode 2 words -> 1 word
# sequences = list()
# for i in range(2, len(encoded)):
# 	sequence = encoded[i-2:i+1]
# 	sequences.append(sequence)
# print('Total Sequences: %d' % len(sequences))



Vocabulary Size: 51173


In [52]:
len(sequences)

19957

In [53]:
sequences

[[515,
  3,
  705,
  22,
  8,
  1182,
  83,
  205,
  234,
  194,
  14,
  1,
  63,
  27487,
  59,
  1182,
  1,
  1340,
  234,
  1980,
  22,
  1,
  213,
  277,
  205,
  308,
  5506,
  4,
  59,
  3545,
  3,
  1448,
  1430,
  4,
  24,
  44,
  1910,
  49,
  1,
  205,
  380,
  8,
  108,
  1182,
  26,
  138,
  713,
  21,
  1209],
 [47,
  3,
  623,
  16,
  1792,
  22,
  26,
  202,
  78,
  274,
  94,
  55,
  184,
  3,
  434,
  1077,
  66,
  24,
  163,
  661,
  471,
  200,
  5683,
  59,
  2209,
  215,
  1158,
  37,
  8,
  210,
  14,
  1,
  212,
  3,
  196,
  3488,
  114,
  1,
  496,
  44,
  13,
  5684,
  8904,
  1792,
  124,
  24,
  166,
  69,
  7,
  2,
  286,
  23,
  24,
  2570,
  134,
  3,
  8,
  2156,
  2009,
  49,
  1,
  120,
  147,
  226,
  29,
  9,
  7,
  5,
  271,
  4284,
  13,
  34,
  20,
  7615,
  14,
  496,
  9,
  28,
  11828,
  99,
  21,
  4,
  1,
  223,
  48,
  328,
  30,
  46,
  2103,
  204,
  91,
  20333,
  8,
  5507,
  135,
  18,
  15,
  13,
  328,
  49,
  184,
  3,
  1191,
  4,
 

In [54]:
len(sequences)

19957

In [55]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
max_length

6229

In [105]:
sequences = pad_sequences(sequences, maxlen=2000, padding='pre')
print('Max Sequence Length: %d' % max_length)


Max Sequence Length: 6229


In [106]:
# split into input and output elements
sequences = array(sequences)
X = sequences
y = y_train

In [107]:
X.shape, y.shape

((19957, 2000), (19957,))

In [108]:
y

array([81, 81, 90, ..., 42, 67, 75])

In [109]:
sequences.shape

(19957, 2000)

In [110]:
# define model
max_words = 10000
model = Sequential()

model.add(Embedding(vocab_size,128))
model.add(LSTM(128))
model.add(Dense(120, activation='linear'))
print(model.summary())


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 128)         6550144   
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 120)               15480     
Total params: 6,697,208
Trainable params: 6,697,208
Non-trainable params: 0
_________________________________________________________________
None


In [113]:
# model.add(Embedding(5, 130))
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam', 
#               metrics=['accuracy'])

# model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 128)         6550144   
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 120)               15480     
_________________________________________________________________
embedding_16 (Embedding)     (None, 120, 128)          640       
_________________________________________________________________
lstm_18 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 129       
_________________________________________________________________
embedding_17 (Embedding)     (None, 1, 130)          

In [ ]:
model.add(Embedding(voca_size,128, 130))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

In [114]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=1, verbose=2)


624/624 - 250s - loss: 5.9185e-06 - accuracy: 0.0107


In [101]:
input = np.array(tokenizer.texts_to_sequences(['text consisting of fewwords']))

In [102]:
padded_input = pad_sequences(input, maxlen=1000, padding='pre')

In [104]:
model.predict_classes(padded_input)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1]], dtype=int32)